In [1]:
import numpy as np
import os
import tempfile

import keras
from keras import backend as K
from keras import layers
from keras.datasets import mnist

import tensorflow as tf

Using TensorFlow backend.


In [2]:
from keras.datasets import mnist
mnist.load_data()

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [3]:
if K.backend() != 'tensorflow':
    raise RuntimeError('This example can only run with the TensorFlow backend,'
                       ' because it requires the Datset API, which is not'
                       ' supported on other platforms.')


def cnn_layers(inputs):
    x = layers.Conv2D(32, kernel_size =(3, 3),
                      activation='relu', padding='valid')(inputs)
    #Keras Conv2D is a 2D Convolution Layer, 
    #this layer creates a convolution kernel that is wind with layers input which helps produce a tensor of outputs.
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    # Max pooling is then used to reduce the spatial dimensions of the output volume.
    
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(512, activation='relu')(x)
     #512 in the hidden layer and 10 neurons=num_classes in the output layer. 
    x = layers.Dropout(0.5)(x)
    predictions = layers.Dense(num_classes,
                               activation='softmax',
                               name='x_train_out')(x)
    return predictions


batch_size = 128 
#The number of steps that are required to complete an epoch is ceil(dataset size/batch size). 
buffer_size = 10000
#if I use the follwing code with 'dataset = dataset.shuffle(buffer_size=2325000) '
#the cost of time to load images is too long for me.


steps_per_epoch = int(np.ceil(60000 / float(batch_size)))  # = 469
#Total number of steps (batches of samples) to yield from generator before declaring one epoch finished and starting the next epoch. 
#It should typically be equal to the number of unique samples of your dataset divided by the batch size.

epochs = 5
#an epoch is a full pass of the dataset. 
#The epoch simply indicates how many times the dataset has been fed to the network.

num_classes = 10

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(np.float32) / 255
x_train = np.expand_dims(x_train, -1)
y_train = tf.one_hot(y_train, num_classes)

# Create the dataset and its associated one-shot iterator.
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.repeat()
dataset = dataset.shuffle(buffer_size)
dataset = dataset.batch(batch_size)
iterator = dataset.make_one_shot_iterator()

# Model creation using tensors from the get_next() graph node.
inputs, targets = iterator.get_next()
model_input = layers.Input(tensor=inputs)
model_output = cnn_layers(model_input)
train_model = keras.models.Model(inputs=model_input, outputs=model_output)

train_model.compile(optimizer=keras.optimizers.RMSprop(lr=2e-3, decay=1e-5),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'],
                    target_tensors=[targets])
train_model.summary()

train_model.fit(epochs=epochs,
                steps_per_epoch=steps_per_epoch)

# Save the model weights.
weight_path = os.path.join(tempfile.gettempdir(), 'saved_wt.h5')
train_model.save_weights(weight_path)

# Clean up the TF session.
K.clear_session()

# Second session to test loading trained model without tensors.
x_test = x_test.astype(np.float32)
x_test = np.expand_dims(x_test, -1)

x_test_inp = layers.Input(shape=x_test.shape[1:])
test_out = cnn_layers(x_test_inp)
test_model = keras.models.Model(inputs=x_test_inp, outputs=test_out)

test_model.load_weights(weight_path)
test_model.compile(optimizer='rmsprop',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])
test_model.summary()

loss, acc = test_model.evaluate(x_test, y_test, num_classes)
print('\nTest accuracy: {0}'.format(acc))

AttributeError: 'BatchDataset' object has no attribute 'make_one_shot_iterator'

-------------------------------------------Practice------------------------------------------------------------------------

In [4]:
#https://keras.io/examples/mnist_dataset_api/
Padding:
#https://www.pyimagesearch.com/2018/12/31/keras-conv2d-and-convolutional-layers/

In [ ]:
x = layers.Conv2D(32, (3, 3),activation='relu', padding='valid')(inputs)

Conv2D is a 2D Convolution Layer, this layer creates a convolution kernel that is wind with layers input which helps produce a tensor of outputs.

a total of 32 filters. Max pooling is then used to reduce the spatial dimensions of the output volume.

Kernel: In image processing kernel is a convolution matrix or masks which can be used for blurring, sharpening, embossing, edge detection, and more by doing a convolution between a kernel and an image.

The kernel_size must be an odd integer as well.

Typical values for kernel_size include: (1, 1) , (3, 3) , (5, 5) , (7, 7) . It’s rare to see kernel sizes larger than 7×7.

ReLU layer(activation='relu'):
ReLU is the abbreviation of rectified linear unit, which applies the non-saturating activation function {\textstyle f(x)=\max(0,x)}{\textstyle f(x)=\max(0,x)}.[54] It effectively removes negative values from an activation map by setting them to zero.[64] It increases the nonlinear properties of the decision function and of the overall network without affecting the receptive fields of the convolution layer.

Padding:(padding='valid'or'same')
A 3×3 kernel applied to an image with padding. The Keras Conv2D padding parameter accepts either "valid" (no padding) or "same" (padding + preserving spatial dimensions).

While the default Keras Conv2D value is valid I will typically set it to same
for the majority of the layers in my network and then either reduce spatial dimensions of my volume by either:

Max pooling Strided convolution

role of the Flatten layer:
    
A flatten operation on a tensor reshapes the tensor to have the shape that is equal to 
the number of elements contained in tensor non including the batch dimension

steps_per_epoch give you the chance to feeding process of the NN when updating the learning rate using ReduceLROnPlateuau(). 
This callback checks the drop of the loss once each epoch has finished and if the loss has stagnated for a patience number of consecutive epochs, 
the callback decreases the learning rate to "slow-cook" the network. 
If your dataset is huge, as it is usually the case when you need to use generators, 
ou would probably like to decay the learning rate within a single epoch (since it includes a big number of data). 
This can be achieved by setting steps_per_epoch to a value that is less than n_samples // batch_size without affecting the overall number of training epochs of your model.

By definition, an epoch is a full pass of the dataset. 
The number of steps that are required to complete an epoch is ceil(dataset size/batch size). 
At each step, #(batch size) samples are fed to the network, a mean loss is calculated and the weights are updated based on that loss. So at each step weights are updated once. 
The epoch simply indicates how many times the dataset has been fed to the network